In [1]:
import numpy as np
import keras as k
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
"""
This function reads all of the files and lables from the json file then opens those files and converts the
images to np arrays to be fed into the nn.
NOTE: this is not a good thing to do if your dataset is too big, this may crash your computer
"""
def imagegen():
    f = open('InfraredSolarModules/module_metadata.json',) 
    data = json.load(f) 
    f.close() 
    img = []
    cat = []
    for i in range(20000):
        img.append(np.array(k.preprocessing.image.load_img('InfraredSolarModules/' + data[str(i)]['image_filepath'], color_mode = "grayscale")))
        cat.append(data[str(i)]['anomaly_class'])
    return np.array(img), np.array(cat)

In [3]:
img, labels = imagegen() # load the data and labels
img = img.reshape(20000, 40, 24, 1) # reshape to show color channels
img = img / 255 # scale data

In [89]:
X_train, X_test, y_train, y_test = train_test_split(img, labels, test_size=0.3)
labelencoder = LabelEncoder() # Make a label encoder 
y_train = labelencoder.fit_transform(y_train) # encode labels to numbers
y_test = labelencoder.transform(y_test)
y_train = k.utils.to_categorical(y_train) # encode nubers to catagorical labels
y_test = k.utils.to_categorical(y_test)

In [70]:
X_train.shape

(14000, 40, 24, 1)

In [71]:
y_train.shape

(14000, 12)

In [80]:
# TODO: Upsample anomalies that don't happen often
counts = {}
for i in y_train:
    try:
        counts[str(i)] += 1
    except:
        counts[str(i)] = 0

In [81]:
counts

{'[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]': 7016,
 '[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]': 1316,
 '[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]': 1051,
 '[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]': 739,
 '[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]': 495,
 '[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]': 1150,
 '[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]': 2319,
 '[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]': 878,
 '[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]': 2599,
 '[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]': 663,
 '[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]': 747,
 '[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]': 547}

In [82]:
for i in range(len(y_train)):
    if counts[str(y_train[i])] < 1000:
        new_X_train = np.array([np.fliplr(X_train[i]), np.flipud(X_train[i]), np.flipud(np.fliplr(X_train[i]))])
        new_y_train = np.array([y_train[i], y_train[i], y_train[i]])
        X_train = np.concatenate((X_train, new_X_train), axis=0)
        y_train = np.concatenate((y_train, new_y_train), axis=0)

In [83]:
X_train.shape

(31757, 40, 24, 1)

In [84]:
y_train.shape

(31757, 12)

In [92]:
model = k.Sequential()
model.add(k.layers.Conv2D(filters=10, kernel_size=(3,3), strides=(1, 1), padding="valid", input_shape=(40, 24, 1)))
model.add(k.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding="same"))
model.add(k.layers.Conv2D(filters=10, kernel_size=(3,3), strides=(1, 1), padding="valid"))
model.add(k.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding="same"))
model.add(k.layers.Flatten())
model.add(k.layers.Dense(450))
model.add(k.layers.Activation('sigmoid'))
model.add(k.layers.Dense(12))
model.add(k.layers.Activation('softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 38, 22, 10)        100       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 19, 11, 10)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 17, 9, 10)         910       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 9, 5, 10)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 450)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 450)               202950    
_________________________________________________________________
activation_39 (Activation)   (None, 450)             

In [93]:
model.fit(X_train, y_train, epochs=20, batch_size=20, verbose=1, shuffle=True)

Epoch 1/20
14000/14000 [==============================] - 6s 414us/step - loss: 1.7600 - accuracy: 0.5038
Epoch 2/20
14000/14000 [==============================] - 6s 402us/step - loss: 1.5585 - accuracy: 0.5348
Epoch 3/20
14000/14000 [==============================] - 6s 398us/step - loss: 1.3390 - accuracy: 0.5997
Epoch 4/20
14000/14000 [==============================] - 6s 401us/step - loss: 1.2283 - accuracy: 0.6295
Epoch 5/20
14000/14000 [==============================] - 6s 400us/step - loss: 1.1642 - accuracy: 0.6456
Epoch 6/20
14000/14000 [==============================] - 6s 399us/step - loss: 1.1072 - accuracy: 0.6606
Epoch 7/20
14000/14000 [==============================] - 6s 395us/step - loss: 1.0587 - accuracy: 0.6735
Epoch 8/20
14000/14000 [==============================] - 6s 396us/step - loss: 1.0112 - accuracy: 0.6862
Epoch 9/20
14000/14000 [==============================] - 6s 399us/step - loss: 0.9638 - accuracy: 0.6981
Epoch 10/20
14000/14000 [=====================

In [94]:
loss, acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Prediction loss, accuracy: {loss, acc}")

6000/6000 [==============================] - 0s 67us/step
Prediction loss, accuracy: (0.848209595521291, 0.7429999709129333)
